In [1]:
# --- Parámetros de rango (ajústalos si quieres) ---
DATE_START = "1945-01-01"
DATE_END   = None  # o "2007-12-01" si quieres replicar el libro

# --- 1) Descargar y alinear datos FRED ---
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pandas_datareader import data as dr

unrate = dr.DataReader("UNRATE", "fred", start=DATE_START, end=DATE_END)  # desempleo mensual (%)
usrec  = dr.DataReader("USREC",  "fred", start=DATE_START, end=DATE_END)  # 1=recesión, 0=no

df = unrate.join(usrec, how="inner").dropna().copy()
df.columns = ["UNRATE", "USREC"]
# Aseguramos 0/1 enteros y un índice mensual “limpio”
df["USREC"] = (df["USREC"] > 0.5).astype(int)
df = df[~df.index.duplicated(keep="first")]

# --- 2) Detectar inicios (0→1) y finales (1→0) de recesión ---
rec = df["USREC"]
start_mask = (rec.shift(1, fill_value=0)==0) & (rec==1)
end_mask   = (rec.shift(1, fill_value=0)==1) & (rec==0)

start_dates = list(df.index[start_mask])
end_dates   = list(df.index[end_mask])

# Manejo de bordes: si empieza/termina dentro de recesión
if len(start_dates)==0 and rec.iloc[0]==1:
    start_dates.insert(0, df.index[0])      # forzamos inicio al primer mes
if len(end_dates)==0 and rec.iloc[-1]==1:
    end_dates.append(df.index[-1])          # forzamos fin al último mes

# Alinear pares (si hay un fin antes del primer inicio, lo eliminamos)
while len(end_dates) and len(start_dates) and end_dates[0] < start_dates[0]:
    end_dates.pop(0)
# Si quedó un inicio sin fin al final, lo quitamos
m = min(len(start_dates), len(end_dates))
start_dates, end_dates = start_dates[:m], end_dates[:m]

# --- 3) Construir tabla pico–valle ---
def months_between(a, b):
    return (b.year - a.year)*12 + (b.month - a.month)

rows = []
prev_trough = None
for s, e in zip(start_dates, end_dates):
    peak = s - pd.offsets.MonthBegin(1)   # pico = mes anterior al inicio de recesión
    trough = e                            # valle = mes en que termina la recesión

    contr_m = months_between(peak, trough)  # duración contracción (pico→valle)
    exp_m = np.nan if prev_trough is None else months_between(prev_trough, peak)

    rows.append({
        "Peak": peak.strftime("%m/%Y"),
        "Trough": trough.strftime("%m/%Y"),
        "ContractionMonths": contr_m,
        "ExpansionMonths": exp_m
    })
    prev_trough = trough

tbl = pd.DataFrame(rows, columns=["Peak","Trough","ContractionMonths","ExpansionMonths"])

# --- 4) Salvaguardas y diagnóstico rápido ---
if tbl.empty:
    print("Diagnóstico rápido:")
    print("Rango:", df.index.min().date(), "→", df.index.max().date())
    print("Valores únicos USREC:", df["USREC"].unique())
    print("Inicios detectados:", len(start_dates), "| Fines detectados:", len(end_dates))
    raise RuntimeError("No se pudieron construir pares pico–valle en el rango elegido. "
                       "Prueba ampliar DATE_START o DATE_END.")

# Promedios (ignora NaN de la primera expansión)
avg_contr = float(pd.to_numeric(tbl["ContractionMonths"], errors="coerce").mean())
avg_exp   = float(pd.to_numeric(tbl["ExpansionMonths"], errors="coerce").mean())
tbl.loc[len(tbl)] = ["Average", "", round(avg_contr,1), round(avg_exp,1)]

tbl.to_csv("nber_cycles_table.csv", index=False)
print("Tabla guardada: nber_cycles_table.csv")
print(tbl.tail(5))

# --- 5) Gráfico: desempleo con recesiones sombreadas ---
fig, ax = plt.subplots(figsize=(9,4))
ax.plot(df.index, df["UNRATE"], linewidth=1.2)
ax.set_ylabel("Unemployment rate (%)")
ax.set_title("U.S. unemployment rate with NBER recessions")

in_rec, start = False, None
for i, (dt, flag) in enumerate(df["USREC"].items()):
    if not in_rec and flag==1:
        in_rec, start = True, dt
    elif in_rec and flag==0:
        ax.axvspan(start, dt, alpha=0.2)   # sombreado sin fijar color
        in_rec = False
if in_rec:  # si termina en recesión
    ax.axvspan(start, df.index[-1], alpha=0.2)

fig.tight_layout()
fig.savefig("us_unemployment_nber.png", dpi=200)
plt.close(fig)
plt.show()
print("Gráfico guardado: us_unemployment_nber.png")


Tabla guardada: nber_cycles_table.csv
       Peak   Trough  ContractionMonths  ExpansionMonths
8   07/1990  04/1991                9.0             91.0
9   03/2001  12/2001                9.0            119.0
10  12/2007  07/2009               19.0             72.0
11  02/2020  05/2020                3.0            127.0
12  Average                        11.3             65.6
Gráfico guardado: us_unemployment_nber.png
